# Chapter 2_1 case1 数据整合
----
Author: W.Y.  
Date: 2020/2/7  
Update: 2020/2/7  
Info: 张俊妮老师书的第二章，数据理解和数据准备，case1，数据整合  

----

In [13]:
# 导入模块

import pandas as pd


In [32]:
# 导入数据

product_a = pd.read_csv('E:\Data Mining\SimplePython\Chapter 2 从R到Python 之 数据挖掘与应用\CH 2_2 数据理解和数据准备\CH 2_2 data ProductAPurchase.csv',
                        dtype={'Date': str, 'AccountNo': str, 'Amount': float})
product_b = pd.read_csv('E:\Data Mining\SimplePython\Chapter 2 从R到Python 之 数据挖掘与应用\CH 2_2 数据理解和数据准备\CH 2_2 data ProductBPurchase.csv', 
                        dtype={'Date': str, 'AccountNo': str, 'Amount': float})
demographic = pd.read_csv('E:\Data Mining\SimplePython\Chapter 2 从R到Python 之 数据挖掘与应用\CH 2_2 数据理解和数据准备\CH 2_2 data Demographics.csv', 
                          dtype={'AccountNo': str, 'Age': float, 'Gender': str})

# dtype 设置每列的格式

In [33]:
# 将Date从字符串格式转为日期格式

product_a['Date'] = pd.to_datetime(product_a['Date'])
product_b['Date'] = pd.to_datetime(product_b['Date'])

In [34]:
product_a['YearMonth'] = product_a['Date'].map(lambda x: 100*x.year + x.month)
product_b['YearMonth'] = product_b['Date'].map(lambda x: 100*x.year + x.month)

# 提取年份和月份，合并为一个变量。如果只要年份或者月份，可以用 product_a['year'] = product_a['Date'].dt.year

In [ ]:
product_a['product'] = 'A'
product_b['product'] = 'B'

In [36]:
product = pd.concat([product_a, product_b]).reset_index(drop=True)

In [37]:
product_sum = product.groupby(['AccountNo', 'product', 'YearMonth'])['Amount'].sum()
product_freq = product.groupby(['AccountNo', 'product', 'YearMonth'])['Amount'].count()

In [38]:
product_final = pd.concat([product_sum, product_freq], axis=1)
product_final.columns = ['sum', 'freq']

In [39]:
product_T = product_final.unstack(['product', 'YearMonth'])
product_T = product_T.reset_index()

In [40]:
product_T.columns = ['AccountNo', '200601 sumA', '200602 sumA', '200601 sumB', '200602 sumB',
                     '200601 freqA', '200602 freqA', '200601 freqB', '200602 freqB']

In [41]:
demographic = demographic.set_index(['AccountNo'])
product_T = product_T.set_index(['AccountNo'])

In [42]:
result = pd.merge(demographic, product_T, how='outer',left_index=True,right_index=True)

In [43]:
result = result.reset_index()
result

# 这里就结束了，如果想把列的顺序调整和书上一样，可以继续一步

,AccountNo,Age,Gender,200601 sumA,200602 sumA,200601 sumB,200602 sumB,200601 freqA,200602 freqA,200601 freqB,200602 freqB
0,001,25.0,F,273.0,371.0,325.0,225.0,2,3,1,2
1,002,32.0,M,50.0,433.0,394.0,270.0,1,2,3,2


In [44]:
order = ['AccountNo', 'Age', 'Gender', '200601 freqA', '200601 sumA', '200602 freqA', '200602 sumA',  
         '200601 freqB', '200601 sumB', '200602 freqB', '200602 sumB']
result_order = result[order]
result_order


,AccountNo,Age,Gender,200601 freqA,200601 sumA,200602 freqA,200602 sumA,200601 freqB,200601 sumB,200602 freqB,200602 sumB
0,001,25.0,F,2,273.0,3,371.0,1,325.0,2,225.0
1,002,32.0,M,1,50.0,2,433.0,3,394.0,2,270.0
